In [ ]:
import theano

get_ipython().magic(u"cd '/home/hartmank/braindecode/convvisual'")
import numpy as np
get_ipython().magic(u'load_ext autoreload')
get_ipython().magic(u'autoreload 2')
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from matplotlib import cm

import matplotlib.lines as mlines
import seaborn
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
matplotlib.rcParams['font.size'] = 7
seaborn.set_style('darkgrid')

from braindecode.scripts.print_results import ResultPrinter
from braindecode.csp.print_results import CSPResultPrinter
import logging
log = logging.getLogger()
log.setLevel("DEBUG")
from braindecode.scripts.train_experiments import setup_logging
setup_logging()

from braindecode.experiments.load import load_exp_and_model
from braindecode.veganlasagne.layer_util import print_layers, recompute_bnorm_layer_statistics
import lasagne

import os
import pickle

In [ ]:
n_Units = 80
n_filters = 5

modelpath = 'models/paper/ours/cnt/deep4/243'
#modelpath = 'models/paper/ours/cnt/deep4/car/22'
savepath = os.path.join(modelpath,'%dBestInputsFor%dBestFilters_FilterActDiff/'%(n_Units,n_filters))
exp, model = load_exp_and_model(os.path.join('../data/',modelpath), set_invalid_to_NaN=False)
get_ipython().magic(u'cd ../')

In [ ]:
from braindecode.veganlasagne.layers import FinalReshapeLayer
print model
all_l = lasagne.layers.get_all_layers(model)
model = all_l[-3]
model = FinalReshapeLayer(model,remove_invalids=True,flatten=False)
model = lasagne.layers.DimshuffleLayer(model,(0,2,1,3))
model = lasagne.layers.ReshapeLayer(model,(-1,4))
model = lasagne.layers.NonlinearityLayer(model,nonlinearity=lasagne.nonlinearities.softmax)

In [ ]:
print_layers(model)

In [ ]:
get_ipython().magic(u"cd '/home/hartmank/braindecode'")
exp.dataset.load()

In [ ]:
ls ..

In [ ]:
#print 'loaded'
datasets = exp.dataset_provider.get_train_merged_valid_test(exp.dataset)
#print 'loaded2'
#inputs,targets = datasets['test'].get_data()
exp.iterator.batch_size = 320
test_batches = list(exp.iterator.get_batches(datasets['test'], shuffle=False))
inputs,targets = test_batches[0]

targets = targets.reshape((len(inputs),-1,4))
targets = targets.mean(axis=1)

In [ ]:

#from braindecode.veganlasagne.layers import create_pred_fn
#pred_fn = create_pred_fn(model)
#inputs,targets = test_batches[0]
#preds = pred_fn(inputs)
#print targets[0],preds.shape[0]
#print("Accuracy: {:.2f}%".format(100 * 
#    np.mean(np.argmax(targets, axis=1) == np.argmax(preds, axis=1))))

In [ ]:
#from braindecode.datahandling.batch_iteration import compute_trial_start_end_samples,CntWindowTrialIterator
#trial_iterator = CntWindowTrialIterator(1, 1145, 521,
#            check_preds_smaller_trial_len=True)
#dir(exp.iterator)
#exp.iterator.input_time_length
#len(compute_trial_start_end_samples(targets)[0])

In [ ]:
#test_batches = list(exp.iterator.get_batches(datasets['test'], shuffle=False))
#len(test_batches)
#len(test_batches[5][0])

In [ ]:
len(np.where(np.any(datasets['test'].get_data()[1]!=0,axis=1))[0])

In [ ]:
len(datasets['test'].get_data()[1])

In [ ]:
get_ipython().magic(u"cd '/home/hartmank/braindecode/convvisual'")
from convvisual import *

from braindecode.veganlasagne.layers import create_pred_fn
model = lasagne.layers.get_all_layers(model)

figpath = os.path.join('/home/hartmank/data/convvisual/figures',savepath)
datpath = os.path.join('/home/hartmank/data/convvisual/RF_data',savepath)
print 'a',figpath
if not os.path.isdir(figpath):
    os.makedirs (figpath)
if not os.path.isdir(datpath):
    os.makedirs (datpath)

In [ ]:
layer_list = [3,5,11,17,23,28]
X_reshapes = [None,None,3,9,27,81]
splits = [2,2,4,4,8,8]

layer_list = [28]
X_reshapes = [81]
splits = [8]

class_defs = [[1,0,0,0],
             [0,1,0,0],
             [0,0,1,0],
             [0,0,0,1]]

for layer,X_reshape,split in zip(layer_list,X_reshapes,splits):
    RF_model,RF_layer = check_if_finalreshape_is_needed(model,layer)
    #Get highest input/unit activation combination
    l_RF = receptive_field_build_deconv_layers(RF_model[RF_layer],RF_model[1],
                                                   use_learned_W=False,X_reshape=X_reshape)
    pred_fn = create_pred_fn(RF_model[RF_layer])
    output = pred_fn(list(inputs))
    #output[output<0] = 0
    
    c=0
    for class_def in class_defs:
        class_indeces = np.where(np.all(np.equal(targets,class_def),axis=1))[0]
        mean_filters = output[class_indeces].mean(axis=3).mean(axis=2).mean(axis=0)
        class_indeces = np.where(~np.all(np.equal(targets,class_def),axis=1))[0]
        mean_filters_base = output[class_indeces].mean(axis=3).mean(axis=2).mean(axis=0)
        print 'Got filter indeces'
        print mean_filters.shape,mean_filters.argmax(axis=0)
        diff_filters = mean_filters-mean_filters_base
        max_filters = diff_filters.argsort(axis=0)[::-1][:n_filters]
    
        max_ind = list()
        for filter in max_filters:
            print 'Filter',filter
            max_ind.append(get_most_active_units_in_layer_from_output(output,RF_layer,filter,n_units=n_Units,abs_act=False))
        #del output
        print 'Max filter finished'
        batches = np.array_split(np.arange(len(max_ind)),split)
        
        X_RF_complete = np.array([])
        max_ind = np.asarray(max_ind)
        for batch in batches:
            print 'Batch'
            if len(batch)==0:
                break

            max_ind_ = max_ind[batch]
            max_ind_shape = max_ind_.shape
            max_ind_ = max_ind_.reshape((1,-1,max_ind_shape[2])).squeeze()

            X_RF_complete_,mask = get_receptive_field_masked_inputs(inputs,max_ind_,l_RF)

            X_RF_shape = X_RF_complete_.shape
            X_RF_complete_ = X_RF_complete_.reshape(max_ind_shape[0],-1,X_RF_shape[1],X_RF_shape[2],X_RF_shape[3])
            X_RF_complete_.shape

            X_RF_complete = np.vstack([X_RF_complete, X_RF_complete_]) if X_RF_complete.size else X_RF_complete_

        pkl = open(os.path.join(datpath,'Class%d_Layer%02d.data'%(c,layer)), 'w')
        savedata = {'RF':X_RF_complete, 'Units':max_ind, 'filters':max_filters, 'filter_act':diff_filters}
        pickle.dump(savedata,pkl)
        pkl.close()
        c+=1
        
pkl = open(os.path.join(datpath,'X_Y_Data.data'), 'w')
savedata = {'X':inputs,'Y':targets}
pickle.dump(savedata,pkl)
pkl.close()

In [ ]:
print max_ind.shape

In [24]:
print batches

[array([0]), array([1]), array([2]), array([3]), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)]
